In [8]:
import os
import pymongo
import pandas
db = pymongo.MongoClient(os.environ['MONGO_HOST'])['flirt']

In [ ]:
# Compare total numbers for simulations on various months.
db.passengerFlows.aggregate([{
   '$match': {
       'arrivalAirport': {
           '$in': ['ATL']
       }
   }
}, {
    '$group': {
        '_id': "$simGroup",
        'total': { '$sum': "$estimatedPassengers" }
    }
}])